In [1]:
import pandas as pd
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy.stats import zscore
from matplotlib.colors import LogNorm, Normalize
import os


In [2]:
FS = 4 # E4 sampling rate

SAVE_CWT_FOLDER = '/media/bayesian-posterior/sdc/sensecode_data/all_freq_cwt_data/'

DATA_FOLDER_PATH = '/media/bayesian-posterior/sdc/sensecode_data/'
DATA_FOLDER = os.fsencode(DATA_FOLDER_PATH)

DAYS_ARRAY = np.asarray([0.5, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31])
FREQ_ARRAY = 1 / (DAYS_ARRAY * 24 * 60 * 60) # for morlet
FREQ_NAMES = ["{:.1f}".format(day)+' Day(s)' for day in DAYS_ARRAY]

# PANDAS_RESAMPLE_RATE = "{:.2f}".format(1/4)+'S' # find missing samples (datetime)
PANDAS_RESAMPLE_RATE = 'T'

In [3]:
def plot_signal(df_eda,
                fs,
                label,
                ylim = None):
    
    plt.figure(figsize=(15, 5))
    
    t = df_eda.index.to_series()
    x = df_eda[label]
    xtick_freq = int(fs*60*60*24*7)
    
    plt.plot(range(len(t)), x, label = label)
    plt.legend(loc=1)
    plt.xticks(range(len(t))[::xtick_freq], t[::xtick_freq], rotation='vertical')

    if ylim is not None:
        plt.ylim(ylim)
    plt.xlabel('Date')
    plt.ylabel('EDA [uS]')
    plt.grid()

In [4]:
def remove_artifacts_and_interpolate(df_eda, 
                     fs, 
                     pandas_resample_rate,
                                     clip_rate,
                                     interpolate_method = 'time',
                                     z_score = False):
    
    df_eda_without_artifacts = df_eda.resample(pandas_resample_rate).first(1)
    
    # df_eda_without_artifacts['hour'] = df_eda_without_artifacts.index.to_series().apply(lambda x : x.hour)    
    # df_eda_without_artifacts['eda'] = df_eda_without_artifacts.groupby('hour', group_keys=False)['eda'].apply(lambda x: x.fillna(x.mean()))
    
    df_eda_without_artifacts.fillna(df_eda_without_artifacts['eda'].mean(), inplace=True)
        
    if clip_rate > 0:
        upper_quantile = df_eda_without_artifacts['eda'].quantile(clip_rate)
        print('EDA upper quantile:', upper_quantile)
        df_eda_without_artifacts.loc[df_eda_without_artifacts['eda'] >= upper_quantile, 'eda'] = upper_quantile
    
    if z_score:
        print('z-scored')
        df_eda_without_artifacts['eda'] = zscore(df_eda_without_artifacts['eda'])
        

    df_eda_without_artifacts.rename(columns={"eda": "interpolated_eda"}, inplace = True)
    return df_eda_without_artifacts

In [5]:
def apply_fir(eda: np.ndarray,
              fs,
              cutoff,
              filter_type, # bandpass, lowpass, highpass, bandstop
              transition_band,
              window) -> np.ndarray:
    
    if window == 'hann' or window == 'hamming' or window == 'bartlett':
        M = int(4 * fs / transition_band)
    elif window == 'blackman':
        M = int(6 * fs / transition_band)
    else:
        raise ValueError('Length estimation for this window not implemented')
        
    # print('Using ' + window + ' window for ' + filter_type + ' FIR filter.')
    h = signal.firwin(numtaps = M, 
                      cutoff = cutoff,
                      fs = fs,
                      pass_zero = filter_type,
                      window = window)
    
    eda = np.squeeze(eda)
    return signal.lfilter(h, [1.0], eda) 
    # return signal.filtfilt(h, [1.0], eda) # avoid phase shift of single filter

In [6]:
def morlet_wavelet(eda: np.ndarray, 
                   fs, 
                   freq_arr, 
                   w = 6):
    
    widths = w * fs / (2 * freq_arr * np.pi)
    cwtm = signal.cwt(eda, signal.morlet2, widths, w = w)
    
    return np.abs(cwtm)


In [7]:
for file in os.listdir(DATA_FOLDER):
    
    filename = os.fsdecode(file)
    
    if filename.endswith("worn_left.h5"):
        
        subject = filename.split('_')[0]
        eda_filepath = DATA_FOLDER_PATH+filename
        
        print(eda_filepath, subject)
        
        df_eda = pd.read_hdf(eda_filepath)
        
        # plot_signal(df_eda = df_eda, label = 'eda', fs = FS)
        df_eda_without_artifacts = remove_artifacts_and_interpolate(df_eda = df_eda, 
                                                                    fs = FS, 
                                                                    pandas_resample_rate = PANDAS_RESAMPLE_RATE, 
                                                                    clip_rate = 0, 
                                                                    z_score = False)
        
#         # plot_signal(df_eda = df_eda_without_artifacts, label = 'eda', fs = FS)
#         df_eda_without_artifacts['low_passed_eda'] = apply_fir(eda = df_eda_without_artifacts['interpolated_eda'].to_numpy(), 
#                                      fs = FS, 
#                                      cutoff = 0.1, 
#                                      transition_band = 0.5,
#                                      filter_type = 'lowpass', 
#                                      window = 'hamming')
        
        cwt_matrix = morlet_wavelet(eda = df_eda_without_artifacts['interpolated_eda'].to_numpy(), 
                            fs = FS, 
                            freq_arr = FREQ_ARRAY)
        
        
        for idx, name in enumerate(FREQ_NAMES):
            df_eda_without_artifacts[name] = cwt_matrix[idx, :]
        
        cwt_h5_name = SAVE_CWT_FOLDER + subject + '_cwt.h5'
        df_eda_without_artifacts[FREQ_NAMES].to_hdf(cwt_h5_name, key='df', mode='w')
    

/media/bayesian-posterior/sdc/sensecode_data/SP22_eda_worn_left.h5 SP22
/media/bayesian-posterior/sdc/sensecode_data/SP69_eda_worn_left.h5 SP69
/media/bayesian-posterior/sdc/sensecode_data/SP28_eda_worn_left.h5 SP28
/media/bayesian-posterior/sdc/sensecode_data/SP21_eda_worn_left.h5 SP21
/media/bayesian-posterior/sdc/sensecode_data/SP59_eda_worn_left.h5 SP59
/media/bayesian-posterior/sdc/sensecode_data/SP72_eda_worn_left.h5 SP72
/media/bayesian-posterior/sdc/sensecode_data/SP71_eda_worn_left.h5 SP71
/media/bayesian-posterior/sdc/sensecode_data/SP1_eda_worn_left.h5 SP1
/media/bayesian-posterior/sdc/sensecode_data/SP12_eda_worn_left.h5 SP12
/media/bayesian-posterior/sdc/sensecode_data/SP45_eda_worn_left.h5 SP45
/media/bayesian-posterior/sdc/sensecode_data/SP65_eda_worn_left.h5 SP65
/media/bayesian-posterior/sdc/sensecode_data/SP11_eda_worn_left.h5 SP11
/media/bayesian-posterior/sdc/sensecode_data/SP24_eda_worn_left.h5 SP24
/media/bayesian-posterior/sdc/sensecode_data/SP31_eda_worn_left.h5